# Lab2 ResNest18


## Google Colab

In [8]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# Change working path
import os
path = "/content/drive/MyDrive/Colab Notebooks/DeepLearning/Lab02"
os.chdir(path)
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/DeepLearning/Lab02


## Import Package

In [9]:
# Numerical Operations
import numpy as np

# Pytorch
import torch
import torch.nn as nn             # Neural Network Modual
import torch.nn.functional as F   # Activation Function
import torch.optim as optim       # Optimizer
import torch.utils.data as data   # Batch training
import torchvision
import torchvision.utils as utils
from torchvision import datasets, transforms, models

# Poltting
import matplotlib.pyplot as plt

import os

## Dataset

In [10]:
# dataset path
data_path_train = "./data/training"
data_path_test = "./data/testing"

In [11]:
# data transform, you can add different transform methods and resize image to any size
img_size = 224
transform = transforms.Compose([
                       transforms.Resize((img_size,img_size)),
                       transforms.ToTensor()
                       ])

# build dataset
dataset = datasets.ImageFolder(root=data_path_train,transform=transform)

# spilt your data into train and val
TOTAL_SIZE = len(dataset)
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])

#build dataloader
train_data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=True,  num_workers=4)

#check dataset
print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 1646
    Root location: ./data/training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
{'Baked Potato': 0, 'Crispy Chicken': 1, 'Donut': 2, 'Fries': 3}


## Train Function

In [12]:
#train function
def train(model, criterion, optimizer):
    model.train()
    total_loss = 0.0
    total_correct = 0

    # Iterate over data
    for inputs, labels in train_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize
        loss.backward()
        optimizer.step()

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)

    avg_loss = total_loss / len(train_data_loader)
    accuracy = total_correct.double() / len(train_dataset) * 100

    print('Training Accuracy: {:.4f}% Training Loss: {:.4f}'.format(accuracy, avg_loss))
    return

#validation function
def valid(model, criterion, scheduler):
    model.eval()
    total_loss = 0.0
    total_correct = 0

    # Iterate over data
    for inputs, labels in val_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)

    avg_loss = total_loss / len(val_data_loader)
    accuracy = total_correct.double() / len(val_dataset) * 100
    scheduler.step(avg_loss)

    print('Validation Accuracy: {:.4f}% Validation Loss: {:.4f}'.format(accuracy, avg_loss))
    return accuracy

## Network Model

In [13]:
from torch.nn.modules.activation import ReLU
from torch.ao.nn.quantized.modules import BatchNorm2d
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# build your model here
class ResBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResBlock, self).__init__()
        self.cross = (in_features != out_features)
        stride = 2 if self.cross else 1
        self.hidden1 = nn.Sequential(
            nn.Conv2d(in_features, out_features, 3, stride, 1),     # Conv2d(in_channels, out_channels, kernel_size, stride, padding)
            nn.BatchNorm2d(out_features)
        )
        self.hidden2 = nn.Sequential(
            nn.Conv2d(out_features, out_features, 3, 1, 1),
            nn.BatchNorm2d(out_features)
        )
        self.downsample = nn.Conv2d(in_features, out_features, 1, 2)   # 1x1 convoultion

    def forward(self, x):
        '''Shortcut path'''
        if (self.cross):
            res = self.downsample(x)
        else:
            res = x

        out = F.relu(self.hidden1(x))
        out = self.hidden2(out)
        out = F.relu(out + res)
        return out

class RN18(nn.Module):
    def __init__(self):
        super(RN18, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=5
        )
        self.conv2_x = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2),
            ResBlock(64, 64),
            ResBlock(64, 64)
        )
        self.conv3_x = nn.Sequential(
            ResBlock(64, 128),
            ResBlock(128, 128)
        )
        self.conv4_x = nn.Sequential(
            ResBlock(128, 256),
            ResBlock(256, 256)
        )
        self.conv5_x = nn.Sequential(
            ResBlock(256, 512),
            ResBlock(512, 512),
            nn.AdaptiveAvgPool2d(1)
        )
        self.out = nn.Linear(512, 1000)


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = torch.flatten(x, start_dim=1)
        x = self.out(x)
        return x

#call model
model = RN18()
# print(model)

## Training

In [14]:
####################  implement your optimizer ###################################
## you can use any training methods if you want (ex:lr decay, weight decay.....)
learning_rate = 1e-4
epochs = 200
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
criterion = nn.CrossEntropyLoss()

# start training
model.to(device=device)
acc_best = 0.0

print('--------------start training--------------')
for epoch in range(1, epochs+1):

    print('epoch:', epoch)
    train(model, criterion, optimizer)
    accuracy = valid(model, criterion, scheduler)

    if accuracy > acc_best:
        acc_best = accuracy
        print("model saved")
        # save the model
        torch.save(model, "model_rn18.pth")

--------------start training--------------
epoch: 1
Training Accuracy: 52.1269% Training Loss: 2.7338
Validation Accuracy: 49.0909% Validation Loss: 2.0558
model saved
epoch: 2
Training Accuracy: 68.6023% Training Loss: 0.8679
Validation Accuracy: 48.4848% Validation Loss: 1.3676
epoch: 3
Training Accuracy: 70.0203% Training Loss: 0.7787
Validation Accuracy: 61.2121% Validation Loss: 1.1784
model saved
epoch: 4
Training Accuracy: 75.7596% Training Loss: 0.6465
Validation Accuracy: 66.0606% Validation Loss: 1.0408
model saved
epoch: 5
Training Accuracy: 76.5699% Training Loss: 0.6180
Validation Accuracy: 62.4242% Validation Loss: 1.2145
epoch: 6
Training Accuracy: 81.4990% Training Loss: 0.5157
Validation Accuracy: 67.8788% Validation Loss: 0.9432
model saved
epoch: 7
Training Accuracy: 80.8238% Training Loss: 0.5238
Validation Accuracy: 67.8788% Validation Loss: 0.9249
epoch: 8
Training Accuracy: 85.9554% Training Loss: 0.4019
Validation Accuracy: 71.5152% Validation Loss: 0.7097
model